## Here we define our own reward function that is also a combination of two reward function (Tangent Hyperbolic function and range function)

In [78]:
import pandas as pd
import numpy as np

df = pd.read_csv('file1.csv')

alpha = 0.2
previousValues = []

def rewardget_combined(score, previousValues):
    # reward function 1
    reward_1 = np.tanh((100 - score) / 10) * 50
    
    # reward function 2
    previousValues_mean = sum(previousValues[-10:]) / len(previousValues[-10:]) if previousValues else 0
    if score >= 0 and score <= 19:
        reward_2 = 50 - (previousValues_mean / 10)
    elif score >= 20 and score <= 39:
        reward_2 = 40 - (previousValues_mean / 10)
    elif score >= 40 and score <= 59:
        reward_2 = 30 - (previousValues_mean / 10)
    elif score >= 60 and score <= 79:
        reward_2 = 20 - (previousValues_mean / 10)
    elif score >= 80 and score <= 99:
        reward_2 = 10 - (previousValues_mean / 10)
    elif score == 100:
        reward_2 = 0 - (previousValues_mean / 10)
    elif score >= 101 and score <= 120:
        reward_2 = -10 - (previousValues_mean / 10)
    elif score >= 121 and score <= 140:
        reward_2 = -20 - (previousValues_mean / 10)
    elif score >= 141 and score <= 160:
        reward_2 = -30 - (previousValues_mean / 10)
    elif score >= 161 and score <= 180:
        reward_2 = -40 - (previousValues_mean / 10)
    elif score >= 181 and score <= 200:
        reward_2 = -50 - (previousValues_mean / 10)
    
    # weighted average reward
    reward = alpha * reward_1 + (1 - alpha) * reward_2
    
    return reward

def valuefind(reward, previousValue, prev_score, current_score):
    if abs(current_score - prev_score) > 100:
        currentValue = 2 * reward + alpha * previousValue
    else:
        currentValue = reward + alpha * previousValue
    return currentValue

def send_notification(message):
    return message

currentValues = []
previousValue = 0
prev_score = df.loc[0, 'score']
notification_messages = []

for i in range(len(df)):
    score = df.loc[i, 'score']
    reward = rewardget_combined(score, previousValues)
    currentValue = valuefind(reward, previousValue, prev_score, score)
    currentValues.append(currentValue)
    previousValues.append(currentValue)
    previousValue = currentValue
    prev_score = score
    
    # Check if an attacker query has occurred or not
    if df.loc[i, 'attack'] == 0:
        # Reset previousValues and previousValue
        previousValues = []
        previousValue = 0

    if currentValue < 0:
        message = "Please be cautious."
        notification_messages.append(send_notification(message))
    else:
        message = "Safe Query"
        notification_messages.append(send_notification(message))

df['currentValues'] = currentValues
df['notification_messages'] = notification_messages

df.head(20)

,Unnamed: 0,query,score,attack,currentValues,notification_messages
0,0,Q1,73,0,25.910075,Safe Query
1,1,Q2,38,0,41.999918,Safe Query
2,2,Q3,128,1,-25.926315,Please be cautious.
3,3,Q4,179,1,-45.111155,Please be cautious.
4,4,Q5,173,1,-48.180723,Please be cautious.
5,5,Q6,91,0,8.705986,Safe Query
6,6,Q7,182,1,-49.999998,Please be cautious.
7,7,Q8,200,1,-56.000000,Please be cautious.
8,8,Q9,15,0,97.279998,Safe Query
9,9,Q10,84,0,17.216686,Safe Query


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# Separate input features and output
X = df[['score', 'attack']]
y = df['currentValues']

# Split the dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

model = LinearRegression()
model.fit(X_train, y_train)

# predictions
y_pred = model.predict(X_test)

# Calculate accuracy
mse = mean_squared_error(y_test, y_pred)
accuracy = 1 - (mse / y_test.var())
print("Accuracy:", accuracy)

Accuracy: 0.9038877918398439
